# Day 16
https://adventofcode.com/2018/day/16

In [1]:
import aocd
data = aocd.get_data(year=2018, day=16)

In [2]:
from dataclasses import dataclass
from functools import reduce
from operator import __add__, __mul__, __and__, __or__
from typing import Callable, Tuple
import regex as re

In [3]:
re_tuple = re.compile(r'(\d+)')

In [4]:
@dataclass(frozen=True)
class Operation():
    operator: Callable[[int, int], int]
    immediate: Tuple[bool]

    def operate(self, registers, args):
        a = args[1] if self.immediate[0] else registers[args[1]]
        b = args[2] if self.immediate[1] else registers[args[2]]
        c = args[3]
        return tuple(self.operator(a, b) if i == args[3] else v for i, v in enumerate(registers))

def first(a, b):
    return a

def gt(a, b):
    return 1 if a > b else 0

def eq(a, b):
    return 1 if a == b else 0

operations = {
    'addr': Operation(__add__, (False, False)),
    'addi': Operation(__add__, (False, True)),
    'mulr': Operation(__mul__, (False, False)),
    'muli': Operation(__mul__, (False, True)),
    'banr': Operation(__and__, (False, False)),
    'bani': Operation(__and__, (False, True)),
    'borr': Operation(__or__, (False, False)),
    'bori': Operation(__or__, (False, True)),
    'setr': Operation(first, (False, False)),
    'seti': Operation(first, (True, False)),
    'gtir': Operation(gt, (True, False)),
    'gtri': Operation(gt, (False, True)),
    'gtrr': Operation(gt, (False, False)),
    'eqir': Operation(eq, (True, False)),
    'eqri': Operation(eq, (False, True)),
    'eqrr': Operation(eq, (False, False)),
}

In [5]:
@dataclass(frozen=True)
class Sample():
    before: Tuple[int]
    inputs: Tuple[int]
    after: Tuple[int]
        
    @classmethod
    def from_input_chunk(cls, chunk):
        return cls(*[tuple(map(int, re_tuple.findall(line))) for line in chunk.split('\n')])
    
    @classmethod
    def list_from_input(cls, text):
        return [cls.from_input_chunk(chunk) for chunk in text.split('\n\n')]
    
    def possible_operation(self, operation):
        try:
            return operation.operate(self.before, self.inputs) == self.after
        except IndexError:
            return False
    
    def possible_operations(self, operations):
        return sum(1 for op in operations.values() if self.possible_operation(op))

In [6]:
def find_opcodes(operations, samples):
    opcodes = {}
    possibilities = {
        sample.inputs[0]: {opname for opname, op in operations.items()
                           if sample.possible_operation(op)}
        for sample in samples
    }
    while len(opcodes) < len(operations):
        opcodes.update({
            opcode: next(iter(opnames))
            for opcode, opnames in possibilities.items()
            if len(opnames) == 1
        })
        solved = set(opcodes.values())
        for opcode in possibilities:
            possibilities[opcode] = possibilities[opcode] - solved
    
    return {
        opcode: operations[opname]
        for opcode, opname in opcodes.items()
    }

In [7]:
def run_program(program, operations):
    registers = (0, 0, 0, 0)
    for line in program.split('\n'):
        args = tuple(map(int, re_tuple.findall(line)))
        registers = operations[args[0]].operate(registers, args)
    return registers

In [8]:
sample_data, program = data.split('\n\n\n\n')
samples = Sample.list_from_input(sample_data)
p1 = sum(1 for sample in samples if sample.possible_operations(operations) >= 3)
print('Part 1: {}'.format(p1))

opcodes = find_opcodes(operations, samples)
p2, *other_registers = run_program(program, opcodes)
print('Part 2: {}'.format(p2))

Part 1: 592
Part 2: 557
